In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets 
from torchvision import transforms
from torchvision.utils import save_image
from torch.autograd import Variable


import pandas as pd 
import numpy as np
import os 
import glob
np.random.seed(1337)  # for reproducibility
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import torch.optim as optim
import torch
import torch.nn.functional as nn
import torch.autograd as autograd
import torch.optim as optim
import numpy as np
#import matplotlib.pyplot as plt
#import matplotlib.gridspec as gridspec
import os
from torch.autograd import Variable
from sklearn.utils import shuffle
import random
from keras.datasets import mnist
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras import optimizers

Using TensorFlow backend.


In [2]:
train_lre = pd.read_csv('/home/satishk/lre2.0/ivectors_csv_revised/train_feat_BNF_h5_07Nov_Shreyas.csv')
#train_afds

In [3]:
#train_lre = train_lre.iloc[1000:2000]

In [4]:
train_lre.head()

,0,1,2,3,4,5,6,7,8,9,...,494,495,496,497,498,499,ids,year,data,lang
0,-0.639420,0.345687,-0.517642,-0.736995,1.313009,1.655726,0.615175,0.799342,1.648420,1.314984,...,-2.216082,-1.322349,-0.716935,-3.843975,-1.471226,0.945404,zkllk,LDC2017E22,data,spa-car
1,0.113609,0.738035,0.584858,-0.248514,0.256618,1.060176,-0.416205,0.133669,0.188325,0.805241,...,-0.082178,1.965844,3.525489,0.006793,1.676278,-1.116285,lid05e1_lid00562,LDC2017E22,data,ara-apc
2,2.061305,-0.140602,-0.627102,-0.682902,1.337618,1.261542,-0.651286,0.307042,-0.980716,0.335533,...,0.996222,-3.018732,-1.841219,0.814874,-0.200840,1.180867,fla_0240-a,LDC2017E22,data,spa-car
3,1.958050,-0.608313,-0.113530,0.167277,0.911105,1.116085,-0.847208,-0.041355,-0.542331,0.199113,...,1.487050,-0.549122,-1.254462,1.644782,0.555329,1.159080,fla_0240-a,LDC2017E22,data,spa-car
4,1.842435,-0.417594,-0.427426,-0.485892,1.204338,1.439185,-1.056640,0.111603,-0.404314,-0.307069,...,0.712544,0.296827,-1.858668,0.209770,0.583769,1.398776,fla_0240-a,LDC2017E22,data,spa-car


In [5]:
train_lre.groupby(['langid']).size()

KeyError: 'langid'

In [ ]:
#Choosing a single class
train_lre = train_lre.loc[train_lre['langid'] == 'ara-apc']

In [63]:
val_lre = pd.read_csv('/home/satishk/lre2.0/ivectors_csv_revised/dev_feat_BNF_h5_07Nov_Shreyas.csv')


In [64]:
#val_lre = val_lre.iloc[100:300]

In [65]:
val_lre.head()

,segmentid,language_code,data_source,speech_duration,0,1,2,3,4,5,...,491,492,493,494,495,496,497,498,499,uttid
0,lre17_ntrlosgu.sph,ara-acm,mls14,30,1.697234,0.029428,-0.400756,0.513963,-0.939232,1.500797,...,-1.314428,-0.927694,-0.370424,-0.514735,1.290885,0.688205,-0.494330,-0.053206,-1.330860,lre17_ntrlosgu
1,lre17_moxnwuqe.sph,ara-acm,mls14,10,1.648232,-0.053318,-0.562867,1.035870,-1.577741,1.593584,...,-0.929262,-1.301574,2.034934,-0.226545,-0.198926,-0.116174,0.347923,-0.870801,-2.599601,lre17_moxnwuqe
2,lre17_meesvkxz.sph,ara-acm,mls14,3,1.242829,0.675515,-0.371491,0.534970,-0.246783,0.806262,...,0.691336,0.257988,1.058771,1.018635,-1.929319,-0.307404,-0.486431,-2.839053,-2.704527,lre17_meesvkxz
3,lre17_rqmsmzui.sph,ara-acm,mls14,30,1.226681,0.014810,-0.396915,-0.097507,-0.013574,1.087025,...,1.049862,0.285627,2.385587,0.680073,1.500978,1.660566,-0.370672,-0.924109,0.096676,lre17_rqmsmzui
4,lre17_qgszpuyw.sph,ara-acm,mls14,10,1.411728,-0.119300,0.136256,0.030535,-1.029447,1.227100,...,0.155196,-1.030222,2.933880,-1.417872,-0.227513,0.748810,-0.671044,0.595977,1.722917,lre17_qgszpuyw


In [66]:
X_train = train_lre.drop("langid",axis=1)
y_train = train_lre["langid"]
#y_train_uttid = train_lre["uttid"]

In [67]:
X_val = val_lre.drop(["language_code","uttid","segmentid","data_source","speech_duration"],axis=1)
y_val = val_lre["language_code"]
y_val_segmentid = val_lre["segmentid"]

In [68]:
X_train = X_train.append(X_val, ignore_index=True)
y_train = y_train.append(y_val, ignore_index=True)

In [69]:
le = preprocessing.LabelEncoder()
le.fit(y_train)

LabelEncoder()

In [70]:
le.classes_

array(['ara-acm', 'ara-apc', 'ara-ary', 'ara-arz', 'eng-gbr', 'eng-usg',
       'por-brz', 'qsl-pol', 'qsl-rus', 'spa-car', 'spa-eur', 'spa-lac',
       'zho-cmn', 'zho-nan'], dtype=object)

In [71]:
y_train=le.transform(y_train)

In [72]:
le = preprocessing.LabelEncoder()
le.fit(y_val)

LabelEncoder()

In [73]:
le.classes_

array(['ara-acm', 'ara-apc', 'ara-ary', 'ara-arz', 'eng-gbr', 'eng-usg',
       'por-brz', 'qsl-pol', 'qsl-rus', 'spa-car', 'spa-eur', 'spa-lac',
       'zho-cmn', 'zho-nan'], dtype=object)

In [74]:
y_val_labels = le.transform(y_val)

In [75]:
y_val_labels[0:100]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0])

In [76]:
X_train=X_train.values
X_val=X_val.values

In [77]:
nb_classes = 14

In [78]:
def indices_to_one_hot(data, nb_classes):
    """Convert an iterable of indices to one-hot encoded labels."""
    targets = np.array(data).reshape(-1)
    return np.eye(nb_classes)[targets]

In [79]:
Y_train = indices_to_one_hot(y_train, 14)
#Y_test = indices_to_one_hot(y_test, 14)
Y_val = indices_to_one_hot(y_val_labels, 14)

In [80]:
Y_val

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]])

In [81]:
Y_train

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]])

In [82]:
Y_val.shape

(3661, 14)

In [83]:
Y_train.shape

(3661, 14)

In [84]:
X_train = X_train.astype('float32')
#X_test = X_test.astype('float32')
X_val = X_val.astype('float32')

#X_train /= 255
#X_test /= 255
#X_val /= 255

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
#print(X_test.shape[0], 'test samples')
print(X_val.shape[0], 'val samples')


X_train shape: (3661, 500)
3661 train samples
3661 val samples


In [85]:
X_train.shape

(3661, 500)

In [86]:
y_train.shape

(3661,)

In [87]:
#X_train,  y_train = shuffle(X_train, y_train, random_state=0)

In [88]:
X_val.shape

(3661, 500)

In [89]:
y_val_labels

array([ 0,  0,  0, ..., 13, 13, 13])

In [90]:
#X_val,  y_val_labels = shuffle(X_val, y_val_labels, random_state=0)

In [91]:
X_train, X_test, Y_train, y_test = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

In [92]:
#mnist = input_data.read_data_sets('/home/satishk/depy_04_AUG/MNIST_data', one_hot=True)
mb_size = 256
Z_dim = 100
X_dim = 500 #mnist.train.images.shape[1]
y_dim = 14 #mnist.train.labels.shape[1]
h_dim = 128
cnt = 0
lr = 1e-3


In [93]:
X_dim, y_dim

(500, 14)

In [94]:
import torch.nn as nn

In [95]:
#Implementing Generator as Encoder-Decoder pair inspired from DAGAN base paper

#Encoder should be able to take a batch of input(of dim 500-ivector) and be able to produce its representation r

class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.fc1 = nn.Linear(500, 256)
        self.fc2 = nn.Linear(256, 128)
        #self.fc3 = nn.Linear(256, 128)
    def forward(self, x):
        return self.fc2(F.relu(self.fc1(x)))
    
#We will use Decoder as Generator    
#Decoder should be able to take input of dim 128(r) and 100(z),concatenate r and z and 
#produce an output od dim 500-ivector    

class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.fc1 = nn.Linear(228,256 )
        self.fc2 = nn.Linear(256, 500)
        #self.fc3 = nn.Linear(512,500)
    def forward(self, x):
        #inputs = torch.cat([z, x], 1)
        return F.sigmoid(self.fc2(F.relu(self.fc1(x))))

#input to Generator alias Decoder is inputs = torch.cat([z, r], 1)
#Where r is the output of encoder given x i.e., representation of x encoded by encoder


In [96]:
#As we defined the Encoder and Decoder we now Implement the Generator 

class Generator(nn.Module):
                                
    def __init__(self):
        super(Generator, self).__init__()
        self.fc1 = Encoder()
        self.fc2 = Decoder()

    def forward(self, z, x):
        #inputs = torch.cat([z, x], 1)
        return self.fc2(torch.cat([z, self.fc1(x)],1))

In [97]:
#Implementation of Discriminator

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        #self.inputs = torch.cat([z, c], 1)
        self.fc1 = torch.nn.Linear(500, 128)
        self.fc2 = torch.nn.Linear(128,1)
        #self.fc3 = torch.nn.Linear(512,1)
        
    def forward(self, x):
        #inputs = torch.cat([X, c], 1)
        return F.sigmoid(self.fc2(F.relu(self.fc1(x))))


In [98]:
G = Generator()
D = Discriminator()

In [99]:
G.modules

<bound method Module.modules of Generator(
  (fc1): Encoder(
    (fc1): Linear(in_features=500, out_features=256)
    (fc2): Linear(in_features=256, out_features=128)
  )
  (fc2): Decoder(
    (fc1): Linear(in_features=228, out_features=256)
    (fc2): Linear(in_features=256, out_features=500)
  )
)>

In [100]:
D

Discriminator(
  (fc1): Linear(in_features=500, out_features=128)
  (fc2): Linear(in_features=128, out_features=1)
)

In [101]:
ones_label = Variable(torch.ones(mb_size))
zeros_label = Variable(torch.zeros(mb_size))
ones_label_fake = Variable(torch.ones(mb_size*2))

In [102]:
learning_rate = 0.0002
betas = (0.5, 0.999)

In [103]:
# Loss function
#criterion = torch.nn.BCELoss()

# Optimizers
G_solver = torch.optim.Adam(G.parameters(), lr=learning_rate, betas=betas)
D_solver = torch.optim.Adam(D.parameters(), lr=learning_rate/2, betas=betas)

In [104]:
len(X_train)

2928

In [105]:
from torch.nn.functional import binary_cross_entropy

In [106]:
batch_size = mb_size
# Start training
for epoch in range(100):
    
    

    print('epoch:',epoch)
    #for i in range(XX_train):
    # Build mini-batch dataset
    #batch_size = images.size(0)
    #images = to_var(images.view(batch_size, -1))

    it=0
    while it+batch_size < len(X_train) :
        

        start= it
        end= it + batch_size


        z = Variable(torch.randn(mb_size, Z_dim))
        X = X_train[start:end]

        #c = Y_train[start:end]
        X = Variable(torch.from_numpy(X))
        #c = Variable(torch.from_numpy(c.astype('float32')))

        # Dicriminator forward-loss-backward-update
        G_sample = G(z, X)
        D_real = D(X)
        D_fake = D(G_sample)

        D_loss_real = binary_cross_entropy(D_real, ones_label)
        D_loss_fake = binary_cross_entropy(D_fake, zeros_label)
        D_loss = D_loss_real + D_loss_fake

        D_loss.backward()
        D_solver.step()

        # Housekeeping - reset gradient
        D.zero_grad()

        # Generator forward-loss-backward-update
        z = Variable(torch.randn(mb_size, Z_dim))
        G_sample = G(z, X)
        X_fake = torch.cat([G_sample, X], 0)
        D_fake = D(X_fake) #Here we need to give Xi along with Xg(i.e. Xg+X or G_sample+X)

        G_loss = binary_cross_entropy(D_fake, ones_label_fake)

        G_loss.backward()
        G_solver.step()

        # Housekeeping - reset gradient
        D.zero_grad()

        #Print and plot every now and then
        #if it % 2 == 0:

        print('Iter-{}; D_loss: {}; G_loss: {}'.format(it, D_loss.data.numpy(), G_loss.data.numpy()))



        it+= batch_size
           

epoch: 0
Iter-0; D_loss: [ 1.33553648]; G_loss: [ 0.77732545]
Iter-256; D_loss: [ 1.30169368]; G_loss: [ 0.79999554]
Iter-512; D_loss: [ 1.25585699]; G_loss: [ 0.81402481]


/home/satishk/miniconda3/envs/lre17/lib/python3.6/site-packages/torch/nn/functional.py:1168: UserWarning: Using a target size (torch.Size([256])) that is different to the input size (torch.Size([256, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))
/home/satishk/miniconda3/envs/lre17/lib/python3.6/site-packages/torch/nn/functional.py:1168: UserWarning: Using a target size (torch.Size([512])) that is different to the input size (torch.Size([512, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


Iter-768; D_loss: [ 1.21911371]; G_loss: [ 0.83304316]
Iter-1024; D_loss: [ 1.18639708]; G_loss: [ 0.85274649]
Iter-1280; D_loss: [ 1.15183926]; G_loss: [ 0.87091476]
Iter-1536; D_loss: [ 1.13988137]; G_loss: [ 0.89904022]
Iter-1792; D_loss: [ 1.11011243]; G_loss: [ 0.91700333]
Iter-2048; D_loss: [ 1.07049179]; G_loss: [ 0.92865592]
Iter-2304; D_loss: [ 1.04459429]; G_loss: [ 0.94536954]
Iter-2560; D_loss: [ 1.03965414]; G_loss: [ 0.97067028]
epoch: 1
Iter-0; D_loss: [ 0.99649155]; G_loss: [ 0.9735446]
Iter-256; D_loss: [ 0.99191022]; G_loss: [ 0.99387091]
Iter-512; D_loss: [ 0.95859206]; G_loss: [ 0.99629956]
Iter-768; D_loss: [ 0.94454336]; G_loss: [ 1.00704563]
Iter-1024; D_loss: [ 0.93085539]; G_loss: [ 1.01571858]
Iter-1280; D_loss: [ 0.9162668]; G_loss: [ 1.02276421]
Iter-1536; D_loss: [ 0.91940081]; G_loss: [ 1.03694785]
Iter-1792; D_loss: [ 0.90667689]; G_loss: [ 1.04288018]
Iter-2048; D_loss: [ 0.88289022]; G_loss: [ 1.04274213]
Iter-2304; D_loss: [ 0.8728236]; G_loss: [ 1.048

Iter-1792; D_loss: [ 0.27676672]; G_loss: [ 1.21630204]
Iter-2048; D_loss: [ 0.26216698]; G_loss: [ 1.21661377]
Iter-2304; D_loss: [ 0.25955397]; G_loss: [ 1.22293162]
Iter-2560; D_loss: [ 0.26777136]; G_loss: [ 1.23446238]
epoch: 14
Iter-0; D_loss: [ 0.24537048]; G_loss: [ 1.23087752]
Iter-256; D_loss: [ 0.27437285]; G_loss: [ 1.25253999]
Iter-512; D_loss: [ 0.23701312]; G_loss: [ 1.24143851]
Iter-768; D_loss: [ 0.24469703]; G_loss: [ 1.252635]
Iter-1024; D_loss: [ 0.24523732]; G_loss: [ 1.26016581]
Iter-1280; D_loss: [ 0.24937242]; G_loss: [ 1.26942325]
Iter-1536; D_loss: [ 0.24084347]; G_loss: [ 1.27221727]
Iter-1792; D_loss: [ 0.24592242]; G_loss: [ 1.2818172]
Iter-2048; D_loss: [ 0.23176652]; G_loss: [ 1.28178883]
Iter-2304; D_loss: [ 0.22941685]; G_loss: [ 1.28769159]
Iter-2560; D_loss: [ 0.23805198]; G_loss: [ 1.29888654]
epoch: 15
Iter-0; D_loss: [ 0.21704008]; G_loss: [ 1.29544544]
Iter-256; D_loss: [ 0.24488129]; G_loss: [ 1.3160013]
Iter-512; D_loss: [ 0.20980944]; G_loss: [

Iter-2560; D_loss: [ 0.08378975]; G_loss: [ 1.8787086]
epoch: 27
Iter-0; D_loss: [ 0.07184071]; G_loss: [ 1.8765049]
Iter-256; D_loss: [ 0.0871416]; G_loss: [ 1.8876704]
Iter-512; D_loss: [ 0.06986267]; G_loss: [ 1.88280761]
Iter-768; D_loss: [ 0.07665399]; G_loss: [ 1.88984501]
Iter-1024; D_loss: [ 0.07756819]; G_loss: [ 1.89394486]
Iter-1280; D_loss: [ 0.07956991]; G_loss: [ 1.89850283]
Iter-1536; D_loss: [ 0.07352354]; G_loss: [ 1.89902341]
Iter-1792; D_loss: [ 0.08061391]; G_loss: [ 1.90605593]
Iter-2048; D_loss: [ 0.07084855]; G_loss: [ 1.90476465]
Iter-2304; D_loss: [ 0.06957343]; G_loss: [ 1.90776801]
Iter-2560; D_loss: [ 0.07841022]; G_loss: [ 1.91566026]
epoch: 28
Iter-0; D_loss: [ 0.06692711]; G_loss: [ 1.91354525]
Iter-256; D_loss: [ 0.08150792]; G_loss: [ 1.92423141]
Iter-512; D_loss: [ 0.06509173]; G_loss: [ 1.91965246]
Iter-768; D_loss: [ 0.07169279]; G_loss: [ 1.92644513]
Iter-1024; D_loss: [ 0.07262332]; G_loss: [ 1.93041933]
Iter-1280; D_loss: [ 0.07443468]; G_loss: [ 

Iter-512; D_loss: [ 0.03268298]; G_loss: [ 2.26170564]
Iter-768; D_loss: [ 0.03727275]; G_loss: [ 2.26649308]
Iter-1024; D_loss: [ 0.03830585]; G_loss: [ 2.26949835]
Iter-1280; D_loss: [ 0.03865736]; G_loss: [ 2.27213287]
Iter-1536; D_loss: [ 0.03452016]; G_loss: [ 2.27249384]
Iter-1792; D_loss: [ 0.04039245]; G_loss: [ 2.2778132]
Iter-2048; D_loss: [ 0.0334997]; G_loss: [ 2.27685738]
Iter-2304; D_loss: [ 0.03267001]; G_loss: [ 2.27898097]
Iter-2560; D_loss: [ 0.03909652]; G_loss: [ 2.28458881]
epoch: 41
Iter-0; D_loss: [ 0.03200548]; G_loss: [ 2.28356147]
Iter-256; D_loss: [ 0.04039101]; G_loss: [ 2.2901206]
Iter-512; D_loss: [ 0.03108673]; G_loss: [ 2.2879653]
Iter-768; D_loss: [ 0.0355415]; G_loss: [ 2.29261851]
Iter-1024; D_loss: [ 0.03657472]; G_loss: [ 2.29556513]
Iter-1280; D_loss: [ 0.03685643]; G_loss: [ 2.29807663]
Iter-1536; D_loss: [ 0.03283458]; G_loss: [ 2.29844904]
Iter-1792; D_loss: [ 0.03860277]; G_loss: [ 2.30365729]
Iter-2048; D_loss: [ 0.03188303]; G_loss: [ 2.30270

Iter-1792; D_loss: [ 0.02439437]; G_loss: [ 2.5449295]
Iter-2048; D_loss: [ 0.01935064]; G_loss: [ 2.54433155]
Iter-2304; D_loss: [ 0.01880373]; G_loss: [ 2.54601455]
Iter-2560; D_loss: [ 0.02326893]; G_loss: [ 2.55012441]
epoch: 54
Iter-0; D_loss: [ 0.01866313]; G_loss: [ 2.54976702]
Iter-256; D_loss: [ 0.02406713]; G_loss: [ 2.55431485]
Iter-512; D_loss: [ 0.01805409]; G_loss: [ 2.55323243]
Iter-768; D_loss: [ 0.02115223]; G_loss: [ 2.55667353]
Iter-1024; D_loss: [ 0.0220972]; G_loss: [ 2.55902481]
Iter-1280; D_loss: [ 0.02185854]; G_loss: [ 2.5607543]
Iter-1536; D_loss: [ 0.01903457]; G_loss: [ 2.56120753]
Iter-1792; D_loss: [ 0.02354806]; G_loss: [ 2.56526709]
Iter-2048; D_loss: [ 0.01861956]; G_loss: [ 2.56467438]
Iter-2304; D_loss: [ 0.01808928]; G_loss: [ 2.56633186]
Iter-2560; D_loss: [ 0.02243304]; G_loss: [ 2.57035327]
epoch: 55
Iter-0; D_loss: [ 0.01796747]; G_loss: [ 2.57001257]
Iter-256; D_loss: [ 0.02321147]; G_loss: [ 2.57445598]
Iter-512; D_loss: [ 0.01737485]; G_loss: 

Iter-0; D_loss: [ 0.01193225]; G_loss: [ 2.7856977]
Iter-256; D_loss: [ 0.01570341]; G_loss: [ 2.78903985]
Iter-512; D_loss: [ 0.01149199]; G_loss: [ 2.78843975]
Iter-768; D_loss: [ 0.01374506]; G_loss: [ 2.79105544]
Iter-1024; D_loss: [ 0.01455577]; G_loss: [ 2.79294658]
Iter-1280; D_loss: [ 0.01415039]; G_loss: [ 2.7942071]
Iter-1536; D_loss: [ 0.0121122]; G_loss: [ 2.79464698]
Iter-1792; D_loss: [ 0.01562694]; G_loss: [ 2.79781961]
Iter-2048; D_loss: [ 0.0119248]; G_loss: [ 2.79745507]
Iter-2304; D_loss: [ 0.01156259]; G_loss: [ 2.79879498]
Iter-2560; D_loss: [ 0.01461897]; G_loss: [ 2.80177259]
epoch: 68
Iter-0; D_loss: [ 0.01157038]; G_loss: [ 2.80176783]
Iter-256; D_loss: [ 0.01524737]; G_loss: [ 2.80504489]
Iter-512; D_loss: [ 0.01113972]; G_loss: [ 2.80447245]
Iter-768; D_loss: [ 0.01334243]; G_loss: [ 2.80703878]
Iter-1024; D_loss: [ 0.01414126]; G_loss: [ 2.80890632]
Iter-1280; D_loss: [ 0.01373129]; G_loss: [ 2.8101182]
Iter-1536; D_loss: [ 0.01174088]; G_loss: [ 2.81057549]

Iter-1536; D_loss: [ 0.00858878]; G_loss: [ 2.93939614]
Iter-1792; D_loss: [ 0.0113748]; G_loss: [ 2.94219017]
Iter-2048; D_loss: [ 0.00849119]; G_loss: [ 2.94222069]
Iter-2304; D_loss: [ 0.00823733]; G_loss: [ 2.9435823]
Iter-2560; D_loss: [ 0.01044173]; G_loss: [ 2.94613075]
epoch: 81
Iter-0; D_loss: [ 0.00828198]; G_loss: [ 2.94652033]
Iter-256; D_loss: [ 0.01099225]; G_loss: [ 2.94927526]
Iter-512; D_loss: [ 0.00794639]; G_loss: [ 2.94917488]
Iter-768; D_loss: [ 0.00961249]; G_loss: [ 2.95142722]
Iter-1024; D_loss: [ 0.01026777]; G_loss: [ 2.95318675]
Iter-1280; D_loss: [ 0.00985067]; G_loss: [ 2.95430088]
Iter-1536; D_loss: [ 0.00835336]; G_loss: [ 2.95489812]
Iter-1792; D_loss: [ 0.0110914]; G_loss: [ 2.95757508]
Iter-2048; D_loss: [ 0.00826163]; G_loss: [ 2.95751143]
Iter-2304; D_loss: [ 0.00801491]; G_loss: [ 2.95877528]
Iter-2560; D_loss: [ 0.01016825]; G_loss: [ 2.96120906]
epoch: 82
Iter-0; D_loss: [ 0.00806164]; G_loss: [ 2.96152711]
Iter-256; D_loss: [ 0.01071423]; G_loss:

Iter-2304; D_loss: [ 0.00606812]; G_loss: [ 3.08907485]
Iter-2560; D_loss: [ 0.00771587]; G_loss: [ 3.09110308]
epoch: 94
Iter-0; D_loss: [ 0.00612701]; G_loss: [ 3.09153724]
Iter-256; D_loss: [ 0.00820007]; G_loss: [ 3.09374833]
Iter-512; D_loss: [ 0.00586355]; G_loss: [ 3.09379458]
Iter-768; D_loss: [ 0.00716269]; G_loss: [ 3.09563708]
Iter-1024; D_loss: [ 0.00770538]; G_loss: [ 3.0971086]
Iter-1280; D_loss: [ 0.00731521]; G_loss: [ 3.09808159]
Iter-1536; D_loss: [ 0.00615813]; G_loss: [ 3.0986836]
Iter-1792; D_loss: [ 0.00836497]; G_loss: [ 3.1009376]
Iter-2048; D_loss: [ 0.00610983]; G_loss: [ 3.10101128]
Iter-2304; D_loss: [ 0.00592543]; G_loss: [ 3.10215282]
Iter-2560; D_loss: [ 0.00753896]; G_loss: [ 3.10413551]
epoch: 95
Iter-0; D_loss: [ 0.00598455]; G_loss: [ 3.10457301]
Iter-256; D_loss: [ 0.00801758]; G_loss: [ 3.10674787]
Iter-512; D_loss: [ 0.00572572]; G_loss: [ 3.10681057]
Iter-768; D_loss: [ 0.00700172]; G_loss: [ 3.10862565]
Iter-1024; D_loss: [ 0.00753668]; G_loss: [

In [107]:
df_SF=pd.DataFrame()
samples_per_class = 1000
#c = np.zeros(shape=[samples_per_class, y_dim], dtype='float32')
#c[:, np.random.randint(0, 10)] = 1.
for i in range(14):
    #print(i)
    c = np.zeros(shape=[samples_per_class, y_dim], dtype='float32')
    c[:, i] = 1.
    c_df=pd.DataFrame(c)
    df_SF = df_SF.append(c_df,ignore_index = True)
    
    

In [109]:
c_df.shape

(1000, 14)

In [ ]:
df_SF.shape[0]

In [ ]:
c_gen = df_SF.values

In [ ]:
z_gen = Variable(torch.randn(df_SF.shape[0], Z_dim))

In [ ]:
z_gen.data.shape

In [ ]:
c_gen = Variable(torch.from_numpy(c_gen))

In [ ]:
samples = G(z_gen, c_gen)

In [ ]:
samples.data.shape

In [ ]:
#Taking the generated iVectors we will try to check the acc by MLP

In [ ]:
train_X1 = samples.data.numpy()

In [ ]:
type(train_X1)

In [ ]:
train_y1 = c_gen.data.numpy()

In [ ]:
type(train_y1)

In [ ]:
train_y1

In [ ]:
X_train = pd.DataFrame(X_train)
Y_train = pd.DataFrame(Y_train)


In [ ]:
train_X1 = pd.DataFrame(train_X1)
train_y1 = pd.DataFrame(train_y1)

In [ ]:
#train_y

In [ ]:
train_X = X_train.append(train_X1, ignore_index=True)
train_y = Y_train.append(train_y1, ignore_index=True)

In [ ]:
#train_X,  train_y = shuffle(train_X, train_y, random_state=0)

In [ ]:
train_X = train_X.values
train_y = train_y.values

In [ ]:
train_y

In [ ]:
model = Sequential()
model.add(Dense(512, input_dim=500))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_classes))
model.add(Activation('softmax'))

In [ ]:
model.summary()

In [ ]:
adam=optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
best_weights_filepath = '/home/satishk/saved_weights/best_weights_2l_MLP_11.hdf5'
saveBestModel = ModelCheckpoint(best_weights_filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='auto')


In [ ]:
batch_size = 256
nb_epoch=30

In [ ]:
#Checking Accuracy with training+augmented data train_X and train_y are 'train + augmented' data
history = model.fit(train_X, train_y, batch_size=batch_size, epochs=nb_epoch,verbose=2, 
                    validation_data=(X_test , y_test),callbacks=[saveBestModel])

In [ ]:
#Frame label accuracy
score = model.evaluate(X_test, y_test, verbose=0)
print('ERROR after Data Augmentation %:', 1-score[1])